In [23]:
# path = input("Path to the local copy of repository:")
import pathlib
import sys
sys.path.insert(1, pathlib.Path().resolve().parents[1])

from modules import pipeline as pe
from local_modules import yield_pipeline as pipe
from local_modules import yield_preprocessing as prep

import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn import model_selection
from importlib import reload

In [25]:
# path = "C:\\Users\\Ivand\\Repos\\banks\\data"
path = input()

 C:\Users\Ivand\Repos\banks\data


In [26]:
df_banks = pd.read_csv(path + "\\BanksDFPrepared.csv")
df_banks.drop(df_banks.columns[0], axis = 1, inplace = True)
df_bonds = pd.read_csv(path + "\\BondsPrepared.csv")
df_bonds.drop(df_bonds.columns[0], axis = 1, inplace = True)
df_bonds = df_bonds.rename({"cregnum" : "REGN"}, axis = 1)



df = df_bonds.merge(df_banks, how = "left", on = ["REGN", "Year", "Month"])
df_full = df.copy()

df.drop(["ISIN"], axis = 1, inplace = True)

# df = pd.get_dummies(df, columns = ["Moodys", "ExpertRA"], drop_first = True)

df = df[~df.NetIncome.isnull()]
df.loc[df.Coupon.isnull(), "Coupon"] = 0
df.loc[df.Coupon_accum.isnull(), "Coupon_accum"] = 0

url = "https://raw.githubusercontent.com/darkmetrics/banks/main/data/metrics.csv"
df_supplement = pd.read_csv(url, parse_dates=True, index_col='date')

non_overlap = []
for i in list(df_supplement.columns):
    if i not in list(df.columns):
        non_overlap.append(i)

# non_overlap.remove("Moodys")
# non_overlap.remove("ExpertRA")
non_overlap.remove("DefaultIn365Days")
non_overlap.remove("DefaultIn730Days")
non_overlap.remove("DefaultIn10000Days")

non_overlap.append("REGN")
non_overlap.append("Year")
non_overlap.append("Month")

df_supplement = df_supplement[non_overlap]

df_full = df_supplement.merge(df, on = ["REGN", "Year", "Month"], how = "right")

print(prep.return_constant_columns(df_full))

df = prep.return_non_constant_table(df_full)

df = df.replace([-np.inf, np.inf], np.nan)

df['asset_std'] = df.groupby('REGN')['Assets'].\
                         transform(lambda x: x.rolling(12, 1).std())
df['loan_std'] = df.groupby('REGN')['Credits_Me'].\
                        transform(lambda x: x.rolling(12, 1).std())
df['corp_loan_std'] = df.groupby('REGN')['CreditsCorporates_Me'].\
                             transform(lambda x: x.rolling(12, 1).std())

df.Moodys = df.Moodys.map({"Ba1" : 1, "Ba2" : 2, "Ba3" : 3, "Baa3" : 4, "B1" : 5, "B2" : 6, "B3" : 7, "C" : 8, "withdrawn" : 9})
df.ExpertRA = df.ExpertRA.map({"A" : 1, "ruAAA" : 1, "ruAA" : 1, "ruA" : 2, "B" : 2, "ruBBB" : 3, "ruBB" : 3, "ruB" : 3, "C" : 4, "ruC" :4, "no" : 5, "withdrawn" : 5})

df = df[np.abs(df.YTM_ind.mean() - df.YTM_ind) <= df.YTM_ind.std()]

X_and_y_banks = df.REGN
X_and_y_banks = X_and_y_banks.unique()
X_and_y_banks_test = np.random.choice(X_and_y_banks, round(len(X_and_y_banks)/5))

X_and_y_test = df[(df.REGN.isin(X_and_y_banks_test))&(df.Year == 2020)]
X_and_y_train = df[(~df.REGN.isin(X_and_y_banks_test))&(df.Year < 2020)]


X_test = X_and_y_test.drop(['G_spread_interpolated','Indicative_price', 'YTM_ind','RF_interpolated'], axis = 1)
X_train = X_and_y_train.drop(['G_spread_interpolated','Indicative_price', 'YTM_ind','RF_interpolated'], axis = 1)
g_spread_test = X_and_y_test['G_spread_interpolated']
g_spread_train = X_and_y_train['G_spread_interpolated']
price_test = X_and_y_test['Indicative_price']
price_train = X_and_y_train['Indicative_price']
ytm_test = X_and_y_test['YTM_ind']
ytm_train = X_and_y_train['YTM_ind']

X_train.loc[(X_train.Callable != 0)&(X_train.Callable != 1), "Callable"] = 1
X_test.loc[(X_test.Callable != 0)&(X_test.Callable != 1), "Callable"] = 1

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
g_spread_test.reset_index(inplace=True, drop=True)
g_spread_train.reset_index(inplace=True, drop=True)
price_test.reset_index(inplace=True, drop=True)
price_train.reset_index(inplace=True, drop=True)
ytm_test.reset_index(inplace=True, drop=True)
ytm_train.reset_index(inplace=True, drop=True)

['NoMoodys']


#### 1. Outliers are filtered at 1 standard deviation
#### 2. Remaining maturities not filtered
#### 3. No preliminary averaging for year-month-rating


In [11]:
X_train_filter1, ys_train1 = prep.outliers_filter(X_train,
                        [g_spread_train, price_train, ytm_train], 
                        ["G_spread_interpolated", "Indicative_price", "YTM_ind"],
                        "(np.abs(X.YTM_ind.mean() - X.YTM_ind) <= X.YTM_ind.std())")
X_test_filter1, ys_test1 = prep.outliers_filter(X_test,
                        [g_spread_test, price_test, ytm_test], 
                        ["G_spread_interpolated", "Indicative_price", "YTM_ind"],
                        "(np.abs(X.YTM_ind.mean() - X.YTM_ind) <= X.YTM_ind.std())")


With standart pipeline

In [12]:
pe = reload(pe)
hpoptimizer = pe.PipeHPOpt2( modules= pipe.pipe_modules, mode='kfold', n_folds = 4, seed=42, binary = False, verbose = False)
lgb_opt1, best_params1, trials1= hpoptimizer.train(X = X_train_filter1, y = ys_train1["G_spread_interpolated"], space=pipe.pipe_para, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)

gen
here
100%|█████████████████████████████████████████████| 100/100 [19:20<00:00, 11.60s/trial, best loss: 0.00872978046169041]


With sampling of banks

In [13]:
pe = reload(pe)
hpoptimizer = pe.PipeHPOpt3( modules= pipe.pipe_modules, folding_objects = "REGN", mode='objects_kfolds', n_folds = 4, seed=42, binary = False, verbose = False, minibatch_size = 2000)
lgb_opt1, best_params1, trials1= hpoptimizer.train(X = X_train_filter1, y = ys_train1["G_spread_interpolated"], space=pipe.pipe_para, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)

41 14 1000 54
100%|█████████████████████████████████████████████| 100/100 [47:18<00:00, 28.39s/trial, best loss: 0.01495088116726429]


In [14]:
g_spread_test = lgb_opt1.predict(X_test_filter1)
np.abs(g_spread_test - ys_test1["G_spread_interpolated"]).mean()

0.049393037084726005

#### 1. Outliers are filtered at 1 standard deviation
#### 2. Remaining maturities above 62 days
#### 3. No preliminary averaging for year-month-rating

In [15]:
X_train_filter2, ys_train2 = prep.outliers_filter(X_train_filter1,
                        ys_train1, 
                        [],
                        "X.Days_to_maturity >= 62")
X_test_filter2, ys_test2 = prep.outliers_filter(X_test_filter1,
                        ys_test1, 
                        [],
                        "X.Days_to_maturity >= 62")


In [16]:
pe = reload(pe)
hpoptimizer = pe.PipeHPOpt2( modules= pipe.pipe_modules, mode='kfold', n_folds = 4, seed=42, binary = False, verbose = False)
lgb_opt2, best_params2, trials2 = hpoptimizer.train(X = X_train_filter2, y = ys_train2["G_spread_interpolated"], space=pipe.pipe_para, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)

gen
here
100%|████████████████████████████████████████████| 100/100 [30:16<00:00, 18.17s/trial, best loss: 0.008156061835572988]


With standart pipeline

In [17]:
pe = reload(pe)
hpoptimizer = pe.PipeHPOpt3( modules= pipe.pipe_modules, folding_objects = "REGN", mode='objects_kfolds', n_folds = 4, seed=42, binary = False, verbose = False, minibatch_size = 2000)
lgb_opt2, best_params2, trials2= hpoptimizer.train(X = X_train_filter2, y = ys_train2["G_spread_interpolated"], space=pipe.pipe_para, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)

41 14 1000 54
100%|████████████████████████████████████████████| 100/100 [51:26<00:00, 30.86s/trial, best loss: 0.014257920485557423]


With sampling of banks

In [18]:
g_spread_test = lgb_opt2.predict(X_test_filter2)
np.abs(g_spread_test - ys_test2["G_spread_interpolated"]).mean()

0.05378933517351708

#### 1. Outliers are filtered at 1 standard deviation
#### 2. Remaining maturities above 62 days
#### 3. Preliminary averaging for year-month-rating

With standart pipeline

In [19]:
pe = reload(pe)
pipe = reload(pipe)
hpoptimizer = pe.PipeHPOpt2( modules= pipe.pipe_modules, mode='kfold', n_folds = 4, seed=42, binary = False, verbose = False)
lgb_opt3, best_params3, trials3 = hpoptimizer.train(X = X_train_filter2, y = ys_train2["G_spread_interpolated"], space=pipe.pipe_para_averager, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)

gen
here
100%|████████████████████████████████████████████| 100/100 [24:24<00:00, 14.64s/trial, best loss: 0.008860110953885435]


With sampling of banks

In [20]:
pe = reload(pe)
pipe = reload(pipe)
hpoptimizer = pe.PipeHPOpt3( modules= pipe.pipe_modules, folding_objects = "REGN", mode='objects_kfolds', n_folds = 4, seed=42, binary = False, verbose = False, minibatch_size = 2000)
lgb_opt3, best_params3, trials3= hpoptimizer.train(X = X_train_filter2, y = ys_train2["G_spread_interpolated"], space=pipe.pipe_para_averager, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)

41 14 1000 54
100%|███████████████████████████████████████████| 100/100 [1:00:40<00:00, 36.40s/trial, best loss: 0.01406088295618157]


In [21]:
g_spread_test = lgb_opt3.predict(X_test_filter2)
np.abs(g_spread_test - ys_test2["G_spread_interpolated"]).mean()

0.04647981932716634

In [22]:
lgb_opt3

Pipeline(steps=[('IndexReseter',
                 <modules.yield_pipeline.IndexReseter object at 0x0000023EBCD2D820>),
                ('Averagermod',
                 <modules.yield_pipeline.averager object at 0x0000023EBE3EF5E0>),
                ('MeanImp', MeanMedianImputer(imputation_method='mean')),
                ('catboost',
                 <catboost.core.CatBoostRegressor object at 0x0000023EBE522E20>)])

### Predicting price

## Debug

In [ ]:
X1 = pe.fallen_data[0]
pe.fallen_pipeline[0].fit(X1)
X2 = pe.fallen_pipeline[0].transform(X1)
pe.fallen_pipeline[1].fit(X2)
X3 = pe.fallen_pipeline[1].transform(X2)
pe.fallen_pipeline[2].fit(X3, pe.fallen_data[1])
X4 = pe.fallen_pipeline[2].transform(X3)
pe.fallen_pipeline[3].fit(X4)
X5 = pe.fallen_pipeline[3].transform(X4)
pe.fallen_pipeline[4].fit(X5)
X6 = pe.fallen_pipeline[4].transform(X5)

Xt1 = pe.fallen_test
Xt2 = pe.fallen_pipeline[0].transform(Xt1)
Xt3 = pe.fallen_pipeline[1].transform(Xt2)
Xt4 = pe.fallen_pipeline[2].transform(Xt3)
Xt5 = pe.fallen_pipeline[3].transform(Xt4)
Xt6 = pe.fallen_pipeline[4].transform(Xt5)

In [ ]:
from importlib import reload
pe = reload(pe)
hpoptimizer = pe.PipeHPOpt3( modules= pipe.pipe_modules, folding_objects = 'REGN', mode='objects_kfolds', n_folds = 4, seed=42, binary = False, verbose = False, minibatch_size = 3000)
lgb_opt1, best_params1, trials1= hpoptimizer.train(X = X_train, y = g_spread_train, space=pipe.pipe_para, trials=pipe.Trials(), algo=pipe.tpe.suggest, max_evals=100)